#### Configuración de token SAS para la conexión con Azure

In [ ]:
spark.conf.set("fs.azure.account.auth.type.tweetsfiles.dfs.core.windows.net", "SAS")
spark.conf.set("fs.azure.sas.token.provider.type.tweetsfiles.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set("fs.azure.sas.fixed.token.tweetsfiles.dfs.core.windows.net", "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-07-07T22:27:21Z&st=2024-05-04T14:27:21Z&spr=https&sig=hNwdq5gK3xFj")

#### Instalación de dependencias necesarias
* Librería de sentimiento utilizada: https://github.com/sentiment-analysis-spanish/sentiment-spanish/blob/master/README.md

In [ ]:
#%pip install transformers
#%pip install torch
#dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Using cached sentiment_analysis_spanish-0.0.25-py3-none-any.whl (30.0 MB)
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


#### Imports

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col
import pandas as pd

sc = SparkContext.getOrCreate()
#variables con los valores de Azure
filename = dbutils.widgets.get('file_name_origin')
routename = dbutils.widgets.get('save_route_databricks')
destination = dbutils.widgets.get('destination_file')

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-2542129566031017>, line 7
      5 sc = SparkContext.getOrCreate()
      6 #variables con los valores de Azure
----> 7 filename = dbutils.widgets.get('file_name_origin')
      8 routename = dbutils.widgets.get('save_route_databricks')
      9 destination = dbutils.widgets.get('destination_file')
NameError: name 'dbutils' is not defined


In [ ]:
schema = StructType([
    StructField('timestamp', StringType(), True),
    StructField('tweet_id', StringType(), True),
    StructField('content', StringType(), True),
    StructField('username', StringType(), True),
    StructField('followers_count', StringType(), True),
    StructField('location', StringType(), True),
    StructField('language', StringType(), True)
])

#### Crear un dataframe de spark y pandas a partir del archivo de origen

In [ ]:
tweets_df = spark.read.format('csv').options(header='true', delimiter=',').schema(schema).load(filename)
tweets_pd = tweets_df.toPandas()
tweets_pd.head(10)

#### Proceso de limpieza del dataframe (dataframe de pandas)

In [ ]:
# Convertir timestamp a datetime y eliminar duplicados y nulos
tweets_pd['timestamp'] = pd.to_datetime(tweets_pd['timestamp'], errors='coerce')
tweets_pd.dropna(subset=['timestamp', 'tweet_id', 'content', 'username', 'language'], inplace=True)
tweets_pd.drop_duplicates(subset=['tweet_id'], inplace=True)
tweets_pd = tweets_pd[tweets_pd['language'] == 'es']
tweets_pd.head(10)

#### Análisis de sentimientos

In [ ]:
from sentiment_analysis_spanish import sentiment_analysis
sentiment = sentiment_analysis.SentimentAnalysisSpanish()
tweets_pd['sentimiento'] = tweets_pd['content'].apply(sentiment.sentiment)
tweets_pd.head(10)

#### Convertir el dataframe a un dataframe de spark

In [ ]:
tweets_df_limpio = spark.createDataFrame(tweets_pd)
tweets_df_limpio.show(10)

#### Guardar el resultado en el destino de Azure

In [ ]:
tweets_df_limpio.write.format('csv').option('header', 'true').save(destination)